<a href="https://colab.research.google.com/github/asyraffatha/Task-MachineLearning/blob/main/Week%2014/Bidirectional_RNN_Model_Asyraff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries
!pip install torch torchvision scikit-learn

# Import libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load and preprocess data
file_path = "/content/drive/MyDrive/Week 14/data.csv"  # Adjust path as needed
data = pd.read_csv(file_path, delimiter=';')

In [4]:
# Clean column names and select numerical data
data.columns = data.columns.str.strip()
data = data.select_dtypes(include=[np.number]).dropna()

In [5]:
# Normalize data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

In [6]:
# Split into features and target
X = data_scaled[:, :-1]
y = data_scaled[:, -1]

In [7]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [9]:
# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [10]:
# Define Bidirectional RNN Model
class BidirectionalRNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, pooling="max"):
        super(BidirectionalRNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True, bidirectional=True)
        self.pooling = pooling
        self.fc = nn.Linear(hidden_size * 2, output_size)  # Multiply by 2 for bidirectional

    def forward(self, x):
        rnn_out, _ = self.rnn(x)
        if self.pooling == "max":
            x = torch.max(rnn_out, dim=1)[0]
        else:
            x = torch.mean(rnn_out, dim=1)
        return self.fc(x)

In [11]:
# Experiment parameters
hidden_sizes = [16, 32, 64]
pooling_types = ["max", "avg"]
epochs_list = [5, 50, 100, 250, 350]
optimizers = ["SGD", "RMSProp", "Adam"]

In [12]:
# Training function
def train_rnn(hidden_size, pooling, optimizer_name, epochs):
    model = BidirectionalRNNModel(input_size=X_train.shape[1], hidden_size=hidden_size, output_size=1, pooling=pooling)
    optimizer = {"SGD": optim.SGD, "RMSProp": optim.RMSprop, "Adam": optim.Adam}[optimizer_name](model.parameters(), lr=0.01)
    criterion = nn.MSELoss()
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    # Training loop
    for epoch in range(epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X.unsqueeze(1))  # Add sequence dimension
            loss = criterion(outputs.squeeze(), batch_y)
            loss.backward()
            optimizer.step()
        scheduler.step()

    return model

In [13]:
# Main experiment
results = []

In [14]:
for hidden_size in hidden_sizes:
    for pooling in pooling_types:
        for optimizer_name in optimizers:
            for epochs in epochs_list:
                model = train_rnn(hidden_size, pooling, optimizer_name, epochs)
                model.eval()
                with torch.no_grad():
                    predictions = []
                    for batch_X, _ in test_loader:
                        pred = model(batch_X.unsqueeze(1))
                        predictions.extend(pred.squeeze().tolist())
                mse_loss = nn.MSELoss()(torch.tensor(predictions), y_test_tensor).item()
                results.append({
                    "hidden_size": hidden_size,
                    "pooling": pooling,
                    "optimizer": optimizer_name,
                    "epochs": epochs,
                    "mse_loss": mse_loss
                })


In [15]:
# Save results
results_df = pd.DataFrame(results)
results_df.to_csv("bidir_rnn_results.csv", index=False)
print("Experiments completed. Results saved to 'bidir_rnn_results.csv'.")

Experiments completed. Results saved to 'bidir_rnn_results.csv'.
